In [2]:
import requests
import lxml.html
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
df = pd.read_csv('test.csv')

In [4]:
index = [x for x in range(0, 7443)]
df1 = pd.DataFrame(index = index, columns = ['bodyType','brand','color','fuelType','image','modelDate','name','numberOfDoors','productionDate','vehicleConfiguration','vehicleTransmission','availability','url','engineDisplacement','enginePower','kmage','Комплектация','fuelType', 'name1', 'name2', 'Привод', 'Руль','Состояние','Владельцы','ПТС','Таможня','Владение','price','priceCurrency'])

In [9]:
i=0
for j in range(1, 201): #200 - кол-во страниц (на одной странице 37 обявлений)


    r = requests.get(f'https://auto.ru/moskva/cars/bmw/used/?output_type=list&page={j}&sort=fresh_relevance_1-desc')
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text, 'html.parser')
    soups1 = soup.findAll('span', itemtype = 'http://schema.org/Car')
    soups2 = soup.findAll('span', itemtype="http://schema.org/Offer")
    soups3 = soup.findAll('span', itemtype="http://schema.org/EngineSpecification")
    soups4 = soup.findAll('div', class_ = 'ListingItem-module__kmAge')
    
    while i<j*len(soups1): 
        for row in soups1[i-((j-1))*len(soups1)]:
            try:
                df1.loc[i][row['itemprop']] = row['content']
            except KeyError:
                continue
                
        for row in soups2[i-((j-1))*len(soups1)]:
            if row['itemprop'] == 'url':
                URL = row['content']
                df1.loc[i][row['itemprop']] = row['content']
            if row['itemprop'] == 'name':
                df1.loc[i]['name1'] = row['content']
            else:
                df1.loc[i][row['itemprop']] = row['content']
    
        try:
            R = requests.get(URL) 
            R.encoding = 'utf-8'
            Soup = BeautifulSoup(R.text, 'html.parser')
        except Exception:
            None
        try:
            #Привод
            df1.loc[i]['Привод'] = Soup.find('li', class_="CardInfo__row CardInfo__row_drive").findAll('span', class_='CardInfo__cell')[1].text
        except AttributeError:
            None
        try:    
            #Руль
            df1.loc[i]['Руль'] = Soup.find('li', class_="CardInfo__row CardInfo__row_wheel").findAll('span', class_='CardInfo__cell')[1].text
        except AttributeError:
            None
        try:    
            #Состояние
            df1.loc[i]['Состояние'] = Soup.find('li', class_="CardInfo__row CardInfo__row_state").findAll('span', class_='CardInfo__cell')[1].text
        except AttributeError:
            None
        try:
            #Владельцы
            df1.loc[i]['Владельцы'] = Soup.find('li', class_="CardInfo__row CardInfo__row_ownersCount").findAll('span', class_='CardInfo__cell')[1].text
        except AttributeError:
            None
        try:
            #ПТС
            df1.loc[i]['ПТС'] = Soup.find('li', class_="CardInfo__row CardInfo__row_pts").findAll('span', class_='CardInfo__cell')[1].text
        except AttributeError:
            None
        try:
            #Таможня
            df1.loc[i]['Таможня'] = Soup.find('li', class_="CardInfo__row CardInfo__row_customs").findAll('span', class_='CardInfo__cell')[1].text
        except AttributeError:   
            None
        try:
            #Владение
            df1.loc[i]['Владение'] = Soup.find('li', class_="CardInfo__row CardInfo__row_owningTime").findAll('span', class_='CardInfo__cell')[1].text
        except AttributeError:   
            None
            
        try:
            #kmage
            df1.loc[i]['kmage'] = Soup.find('li', class_="CardInfo__row CardInfo__row_kmAge").findAll('span', class_='CardInfo__cell')[1].text
        except AttributeError:   
            None       
        
        for row in soups3[i-((j-1))*len(soups1)]:
            if row['itemprop'] == 'name':
                df1.loc[i]['name2'] = row['content']
            else:
                df1.loc[i][row['itemprop']] = row['content']
            
        try:
            card_complectation = []
            for row in Soup.find('div', class_="CardComplectation__groups").findAll('div', class_= 'CardComplectation__group'):
                #print(row)
                for ROW in row.findAll('li', class_='CardComplectation__itemContentEl'):
                    #print(ROW)
                    card_complectation.append(ROW.text)
            #print(card_complectation)        
            df1.loc[i]['Комплектация'] = card_complectation
        except Exception:
            None
            
        i += 1
        print(f'Кол-во спарсенных объявлений: {i}', end="\r")

In [91]:
df1.to_csv('autoru3.csv')